In [17]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, to_timestamp

In [18]:
spark = SparkSession.builder.master("local[*]").getOrCreate()
spark

In [19]:
customer = spark.read.csv(r"C:\Users\sofia\OneDrive\Documentos\7° Semestre\Programación concurrente\customer_profile_dataset.csv", header=True)
customer.show(5)

+-----------+----------+---------+------+-------------------+--------------------+------------+-------------------+-------------+-----------+-----+--------+
|customer_id|first_name|last_name|gender|      date_of_birth|               email|phone_number|        signup_date|      address|       city|state|zip_code|
+-----------+----------+---------+------+-------------------+--------------------+------------+-------------------+-------------+-----------+-----+--------+
|          1|    Robert|    Smith|Female|1994-06-14 21:40:27|jane.davis1@mail.com|634-106-4981|2016-10-16 17:23:25| 8465 Main St|San Antonio|   CA|   35566|
|          2|     Emily|   Garcia|Female|1989-09-21 17:56:31|robert.williams2@...|386-635-5998|2021-04-04 14:24:06|  305 Main St|   New York|   AZ|   23187|
|          3|   Jessica|    Brown|  Male|1984-01-21 21:43:13|emily.davis3@mail...|627-341-5213|2018-04-22 04:51:57|  5725 Oak St|    Chicago|   AZ|   99188|
|          4|   Michael|    Brown|  Male|1986-02-06 13:09:

In [20]:
products = spark.read.csv(r"C:\Users\sofia\OneDrive\Documentos\7° Semestre\Programación concurrente\products_dataset.csv", header=True)
products.show(5)

+----------+------------+--------+--------------+------+--------------------+
|product_id|product_name|category|price_per_unit| brand| product_description|
+----------+------------+--------+--------------+------+--------------------+
|         1|      Butter|   Dairy|         28.58|BrandB|Description for Rice|
|         2|      Butter|   Meats|         22.66|BrandB|Description for B...|
|         3|        Milk|   Meats|         26.52|BrandE|Description for B...|
|         4|      Banana|  Grains|         26.12|BrandB|Description for A...|
|         5|        Rice|  Fruits|         21.94|BrandD|Description for B...|
+----------+------------+--------+--------------+------+--------------------+
only showing top 5 rows



In [21]:
purchase = spark.read.csv(r"C:\Users\sofia\OneDrive\Documentos\7° Semestre\Programación concurrente\purchase_history_dataset.csv", header=True)
purchase.show(5)

+-----------+-----------+----------+-------------------+--------+------------------+
|purchase_id|customer_id|product_id|      purchase_date|quantity|      total_amount|
+-----------+-----------+----------+-------------------+--------+------------------+
|          1|          1|        42|2018-04-15 14:08:01|       3| 37.64207365077783|
|          2|          1|       138|2022-07-10 23:33:47|       4| 70.24710587172727|
|          3|          1|       403|2021-12-31 03:53:33|       3| 89.16889585975464|
|          4|          1|       193|2017-01-14 01:25:11|       2| 59.70505931112876|
|          5|          1|        26|2018-04-06 11:01:06|       3|101.77886387225126|
+-----------+-----------+----------+-------------------+--------+------------------+
only showing top 5 rows



Ejercicios; contesta desarrollando el correspondiente código en Pyspark:

1. Responde, ¿cuántos clientes llamados "Robert" (nota cómo hay *Males* y *Females*), compraron algún producto lácteo (Dairy) en 2022 ?

2. Eres empleado de *BrandB*. ¿En cuáles ciudades has vendido una mayor cantidad? (total_amount)

3. ¿De cuánto es la mayor cantidad (quantity) que ha sido comprado por algún hombre O cuyo producto sea pan (Bread)?

In [24]:
# Realizar un join entre las tablas `purchase`, `customer` y `products`
join_1 = purchase.join(customer, on="customer_id").join(products, on="product_id")

# Filtrar: clientes llamados "Robert", productos de categoría "Dairy" y compras realizadas en 2022
resultado_1 = join_1.filter(
                            (col("first_name") == "Robert") &
                            (col("category") == "Dairy") &
                            (purchase["purchase_date"] > lit("2022-01-01 00:00:00")) &
                            (purchase["purchase_date"] < lit("2022-12-31 23:59:59"))
                        ) \
                    .select("customer_id").distinct().count()
resultado_1

31

In [28]:
# Asegurarnos de que la columna `total_amount` es numérica
purchase = purchase.withColumn("total_amount", col("total_amount").cast("double"))

# Realizar un join entre las tablas `purchase` y `products`
join_2 = purchase.join(products, on="product_id").join(customer, on="customer_id")

# Filtrar productos de la marca "BrandB"
resultado_2 = join_2.filter(col("brand") == "BrandB")\
                    .groupBy("city").sum("total_amount")\
                    .orderBy(col("sum(total_amount)").desc())\

# Mostrar las ciudades con mayor cantidad vendida
resultado_2.show()

+------------+------------------+
|        city| sum(total_amount)|
+------------+------------------+
|     Chicago|29557.642903425036|
|     Houston| 28586.38258782695|
|     Phoenix| 28041.96419969516|
|Philadelphia|27492.074313737838|
|    New York|27246.218068437967|
| San Antonio|25221.051797835673|
| Los Angeles|23819.529899740137|
+------------+------------------+



In [29]:
# Realizar el join entre las tablas purchase, customer y products
join_3 = purchase.join(customer, on="customer_id").join(products, on="product_id")

# Filtrar: compras realizadas por hombres O productos con nombre "Bread"
resultado_3 = join_3.filter((col("gender") == "Male") | 
                            (col("product_name") == "Bread"))\
                    .agg({"quantity": "max"}).collect()[0][0]
resultado_3

'5'